In [1]:
using CUDA, Random
using GPUArrays: @allowscalar
include("utils.jl")
CUDA.allowscalar(false)

In [2]:
### Global variables
S, nx, ny = 4, 4, 3
px, py = partition(nx, ny)
pooled = false
DTYPE = Float32

### Generate standard normal data
Random.seed!(123)
xs = randn(DTYPE, S, nx)
ys = randn(DTYPE, S, ny)
# move to GPU
xs = cu(xs)
ys = cu(ys)
# get first pair
x1 = xs[1,:]
y1 = ys[1,:]
data = vcat(x1, y1)

7-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -0.6457307
  0.49224567
 -1.3416092
 -0.07616795
  0.20607506
  0.26140082
  0.109018035

In [3]:
data[px]  # returns GPU array even if index array is on CPU

35×4 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.645731    0.492246   -1.34161    -0.0761679
 -0.645731    0.492246   -1.34161     0.206075
 -0.645731    0.492246   -1.34161     0.261401
 -0.645731    0.492246   -1.34161     0.109018
 -0.645731    0.492246   -0.0761679   0.206075
 -0.645731    0.492246   -0.0761679   0.261401
 -0.645731    0.492246   -0.0761679   0.109018
 -0.645731    0.492246    0.206075    0.261401
 -0.645731    0.492246    0.206075    0.109018
 -0.645731    0.492246    0.261401    0.109018
 -0.645731   -1.34161    -0.0761679   0.206075
 -0.645731   -1.34161    -0.0761679   0.261401
 -0.645731   -1.34161    -0.0761679   0.109018
  ⋮                                  
  0.492246   -1.34161     0.206075    0.261401
  0.492246   -1.34161     0.206075    0.109018
  0.492246   -1.34161     0.261401    0.109018
  0.492246   -0.0761679   0.206075    0.261401
  0.492246   -0.0761679   0.206075    0.109018
  0.492246   -0.0761679   0.261401    0.109018
  0.492246    0.206

In [4]:
function _mean(x, d)
    d = ndims(x)
    return sum(x, dims=d) ./ size(x, d)
end


function _var(x, d, means)
    nx = size(x, d)
    ss = sum(x.^2, dims=d)
    return @. (ss - (nx * means.^2)) / (nx-1)
end


function _var(x, d)
    means = _mean(x ,d)
    return _var(x, d, means)
end

using Statistics
println(Statistics.var(xs, dims=2))
println(_var(xs, 2))

Float32[0.6159249; 1.1902627; 1.1496912; 1.000899;;]
Float32[0.6159249; 1.1902624; 1.1496912; 1.0008991;;]


In [5]:
function t(xs, ys, pooled)
    """
    Parameters
    ----------
    xs : AbstractArray{Real, N}
        Data for group 1
        If N == 1, then size(xs) = (nx,)
        If N == 2, then size(xs) = (S, nx)
    
    ys : AbstractArray{Real, N}
        Data for group 2
        If N == 1, then size(ys) = (ny,)
        If N == 2, then size(ys) = (S, ny)
    
    pooled : Bool
        Assume equal/unequal variances for the two groups
    
    Returns
    -------
    S-element Vector{Float64}
        t test statistic for each pair
    """
    d = ndims(xs)  # if 1D vector, compute by column, if 2D matrix, compute by row

    meanx = _mean(xs, d)
    varx  = _var(xs, d, meanx)
    
    meany = _mean(ys, d)
    vary = _var(ys, d, meany)

    nx, ny = size(xs, d), size(ys, d)  # number of observations per group

    if pooled
        pooled_var = ((nx-1)*varx + (ny-1)*vary) / (nx+ny-2)
        denom = sqrt.(pooled_var * (1/nx + 1/ny))
    else
        denom = sqrt.(varx/nx + vary/ny)
    end

    return (meanx-meany)./denom
end


println(t(x1, y1, pooled))            # 1D array input
println(t(xs, ys, pooled))            # automatically vectorize for each row
t.(eachrow(xs), eachrow(ys), pooled)  # dot syntax to vectorize for each row

Float32[-1.481249]
Float32[-1.481249; 0.68992394; 0.2269192; 0.047226395;;]


4-element Vector{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}:
 Float32[-1.481249]
 Float32[0.68992394]
 Float32[0.2269192]
 Float32[0.047226395]

In [7]:
function pval(x, y, pooled=false, alternative="two-sided", delta=0)
    """
    Parameters
    ----------
    x : Vector{Real}
        Data for group 1
    
    y : Vector{Real}
        Data for group 2
    
    pooled : Bool
        Assume equal/unequal variances for the two groups
    
    alternative : String
        Type of alternative hypothesis
    
    delta : Real
        Null hypothesis difference in means
    
    Returns
    -------
    Float64
        Proportion of pairs among all sample combinations which have
        a test statistic as or more extreme than the original pair (x, y)
    """
    
    x_shift = x .- delta              # shift group 1 under null hypothesis
    t_obs = t(x_shift, y, pooled)  # test statistic for observed data

    combined = vcat(x_shift, y)  # join original pair into single vector
    xs = combined[px]   # get all combinations of pairs from original pair
    ys = combined[py]
    ts = t(xs, ys, pooled)   # test statistic for all possible pairs of samples
    
    if alternative == "smaller"
        n_extreme = count(ts .<= t_obs)
    elseif alternative == "larger"
        n_extreme = count(ts .>= t_obs)
    else
        n_extreme = count(@. (ts <= -abs(t_obs)) | (ts >= abs(t_obs)))
    end

    return n_extreme / size(px, 1)  # proportion of pairs w/ extreme test statistic
    
    
end


for i in 1:S
    println(pval(xs[i,:], ys[i,:], pooled))  # Must accept 1D array input
end
pval.(eachrow(xs), eachrow(ys), pooled)      # But can be vectorized with dot syntax

0.2857142857142857
0.5142857142857142
0.7428571428571429
0.9714285714285714


4-element Vector{Float64}:
 0.2857142857142857
 0.5142857142857142
 0.7428571428571429
 0.9714285714285714

In [10]:
using Distributions

# TODO consider merging with ttest_ind()
function tconf(x1, x2; pooled=true, alpha=0.05)
    """Returns (1-alpha)% t-test confidence interval for the difference in means.
    Reference: http://www.stat.yale.edu/Courses/1997-98/101/meancomp.htm
    Parameters
    ----------
    x1 : Vector{Int64}
        Data for group 1
    x2 : Vector{Int64}
        Data for group 2
    pooled : Bool
        Assume pooled (equal) or unpooled (unequal) variances
    alpha : Float64
        Significance level
    Returns
    -------
    Tuple{Float64, Float64}
        (1-alpha)% t-test confidence interval for the difference in means
    """
    d1, d2 = ndims(x1), ndims(x2)
    n1, n2 = size(x1, d1), size(x2, d2)
    
    if pooled
        dof = n1 + n2 - 2
        var1 = var2 = ((n1-1).*var(x1, dims=d1) .+ (n2-2).*var(x2, dims=d2)) ./ dof
        #var1, var2 = var(x1, dims=d1), var(x2, dims=d2)
    else
        dof = min(n1, n2) - 1
        var1, var2 = var(x1, dims=d1), var(x2, dims=d2)
    end

    t = TDist(dof)
    tcrit = quantile(t, 1-(alpha/2))
    margin = @. tcrit * sqrt(var1/n1 + var2/n2)
    diff = mean(x1, dims=d1) .- mean(x2, dims=d2)
    return vcat(zip(diff .- margin, diff .+ margin)...)
end

wide   = @allowscalar tconf(xs, ys, alpha=0.0001)
narrow = @allowscalar tconf(xs, ys, alpha=0.3)

wide_lo   = cu([i for (i, _) in wide])
wide_hi   = cu([i for (_, i) in wide])
narrow_lo = cu([i for (i, _) in narrow])
narrow_hi = cu([i for (-, i) in narrow])

4-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -0.047495052
  1.3215078
  0.99111
  0.7441092

In [8]:
function search(x, y, start, stop;
                pooled=false, alternative="two-sided", isLowerBound=true,
                margin=0.005, threshold=1.0, alpha=0.05)
    
    p_start = pval(x, y, pooled, alternative, start)
    p_end   = pval(x, y, pooled, alternative, stop)
    #println("p_start = ", p_start, ", p_end = ", p_end)
    
    # p-values corresponding to `start` and `stop` must be on opposite sides of `alpha`
    @assert (p_start - alpha) * (p_end - alpha) <= 0

    p = p_new = delta = nothing
    percent_change = (old, new) -> 100 * abs(new-old) / old
    
    i = 0
    while true
        i += 1
        delta = (start + stop) / 2
        p_new = pval(x, y, pooled, alternative, delta)

        if !isnothing(p) && percent_change(p, p_new) <= threshold
            break  # (1) percent change in p-value is below `threshold`
        end
        
        compare = (alpha - p_new) - isLowerBound * 2 * (alpha - p_new)
        if margin < compare
            stop = delta
        elseif margin < -compare
            start = delta
        else
            break  # (2) p-value is within `margin` of `alpha`
        end

        p = p_new
    end
    
    return delta
end


#search.(eachrow(xs), eachrow(ys), wide_lo, narrow_lo)

search (generic function with 1 method)

In [21]:
for i in 1:S
    println(@allowscalar search(xs[i,:], ys[i,:], wide_lo[i], narrow_lo[i]))
end

search.(eachrow(xs), eachrow(ys), eachrow(wide_lo), eachrow(narrow_lo))

-2.5789237
-2.415457
-3.3115125
-2.2412906


4-element Vector{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}:
 Float32[-2.5789237]
 Float32[-2.415457]
 Float32[-3.3115125]
 Float32[-2.2412906]